In [1]:
# импортируем необходимые библиотеки, классы SimpleImputer, 
# StandardScaler, OneHotEncoder, ColumnTransformer, 
# LogisticRegression, Pipeline, функции cross_val_score(),
# cross_val_predict(), cross_validate()
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (GradientBoostingRegressor,
                              GradientBoostingClassifier)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (cross_val_score, 
                                     cross_val_predict,
                                     cross_validate)

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head(3)

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No


In [3]:
# создаем массив меток и массив признаков
y = data.pop('Response')

In [4]:
# создаем списки категориальных 
# и количественных столбцов
cat_columns = data.select_dtypes(
    include='object').columns.tolist()
num_columns = data.select_dtypes(
    exclude='object').columns.tolist()

In [5]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [6]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

In [7]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [8]:
# задаем итоговый конвейер
ml_pipe = Pipeline([
    ('transform', transformer), 
    ('logreg', LogisticRegression(solver='lbfgs', 
                                  max_iter=200))])

In [9]:
# выполняем перекрестную проверку, разместив итоговый 
# конвейер внутри цикла перекрестной проверки
cv_mean = cross_val_score(ml_pipe, data, y, cv=5).mean()
print("Усредненная правильность cv: {:.3f}".format(cv_mean))

Усредненная правильность cv: 0.897


In [10]:
# выполняем перекрестную проверку, разместив итоговый 
# конвейер внутри цикла перекрестной проверки
cv_mean = cross_val_score(
    ml_pipe, data, y, scoring='roc_auc', cv=5).mean()
cv_std = cross_val_score(
    ml_pipe, data, y, scoring='roc_auc', cv=5).std()
print("Усредненная оценка AUC-ROC cv: {:.3f}".format(cv_mean))
print("Стандартное отклонение AUC-ROC cv: {:.3f}".format(cv_std))
cv_mean_corr = cv_mean - cv_std
print(f"Усредненная оценка AUC-ROC c поправкой\n" 
      f"на стандартное отклонение: {cv_mean_corr:.3f}")

Усредненная оценка AUC-ROC cv: 0.635
Стандартное отклонение AUC-ROC cv: 0.026
Усредненная оценка AUC-ROC c поправкой
на стандартное отклонение: 0.609


In [11]:
# мы вычисляем спрогнозированные значения для 
# наблюдений каждого тестового блока с помощью
# модели, обученной на обучающих блоках
predictions = cross_val_predict(ml_pipe, data, y, cv=5)
predictions

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [12]:
# создаем объект, в который будем сохранять
# результаты перекрестной проверки
cv_results = cross_validate(ml_pipe, 
                            data, 
                            y, 
                            cv=5, 
                            return_estimator=True)

# создаем пустой список, в который будем сохранять 
# коэффициенты логистической регрессии
coefs = []
# для каждой модели сохраняем константы коэффициенты в список
for model in cv_results['estimator']:    
    coefs.append(list(model[1].intercept_) + list(model[1].coef_[0]))

In [13]:
# извлекаем дамми-переменные, созданные классом OneHotEncoder
ml_pipe.fit(data, y)
cat = ml_pipe.named_steps['transform'].named_transformers_['cat']
onehot_columns = list(cat.named_steps['ohe'].get_feature_names_out(
    input_features=cat_columns))

In [14]:
# добавляем в конец списка количественных переменных 
# дамми-переменные, созданные OneHotEncoder, т.е. 
# сохраняем тот же порядок столбцов, что задал 
# ColumnTransformer
num_columns.extend(onehot_columns)

# добавляем Intercept в начало списка
num_columns.insert(0, 'Intercept')

In [15]:
# теперь сопоставляем признакам коэффициенты модели
lr_coefs = pd.DataFrame(coefs, columns=num_columns).T
# смотрим на результат
lr_coefs

,0,1,2,3,4
Intercept,-1.655754,-1.734709,-1.727187,-1.693281,-1.707346
Customer Lifetime Value,-0.027229,0.016621,-0.004951,-0.057115,-0.048046
Income,0.072515,0.058170,0.033085,0.045221,0.058014
Monthly Premium Auto,0.105276,0.092715,0.139402,0.129031,0.129856
Months Since Last Claim,-0.025576,-0.023675,-0.026496,-0.015031,-0.057444
Months Since Policy Inception,-0.016235,0.007449,-0.014484,0.016502,0.012168
Number of Open Complaints,-0.041276,-0.035427,-0.034442,-0.030048,-0.014053
Number of Policies,-0.058914,-0.051552,-0.053842,-0.046859,-0.090626
Coverage_Basic,-0.029730,0.041536,0.062551,-0.005767,0.014594
Coverage_Extended,0.085790,0.083429,0.090402,0.104599,0.107809


In [16]:
# записываем CSV-файл в объект DataFrame
df = pd.read_csv('Data/Flats_missing.csv', sep=';', decimal=',')
df.head(3)

,Rooms_Number,District,Stor,Storeys,Space_Total,Space_Living,Space_Kitchen,Balcon_Num,Lodgee_Num,lat,Long,Cost_KV
0,1,Заельцовский,13,17.0,54.1,18.0,21.2,0.0,1,55.0725,82.9069,50831.79298
1,1,Заельцовский,10,17.0,54.5,18.0,21.1,0.0,1,55.0725,82.9069,52000.00000
2,1,Центральный,8,17.0,37.0,0.0,0.0,0.0,0,55.0725,82.9068,87837.83784


In [17]:
# создаем массив меток и массив признаков
label = df.pop('Cost_KV')

In [18]:
# создаем списки категориальных 
# и количественных столбцов
cat_cols = df.select_dtypes(
    include='object').columns.tolist()
num_cols = df.select_dtypes(
    exclude='object').columns.tolist()

In [19]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [20]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_cols),
                ('cat', cat_pipe, cat_cols)]

In [21]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [22]:
# задаем итоговый конвейер
pipe = Pipeline([
    ('transform', transformer),
    ('boost', GradientBoostingRegressor(
        max_depth=6, random_state=46))])

In [23]:
# выполняем перекрестную проверку, разместив итоговый 
# конвейер внутри цикла перекрестной проверки
rmse_cv = cross_val_score(pipe, 
                          df, 
                          label, 
                          scoring='neg_mean_squared_error', 
                          cv=5)
r2_cv = cross_val_score(pipe, df, label, cv=5)
print("RMSE, усреднение по 5 тестовым блокам cv: {:.3f}".format(
    np.mean(np.sqrt(np.abs(rmse_cv)))))
print("R2, усреднение по 5 тестовым блокам cv: {:.3f}".format(
    np.mean(r2_cv)))

RMSE, усреднение по 5 тестовым блокам cv: 11864.022
R2, усреднение по 5 тестовым блокам cv: 0.404


In [24]:
# загружаем данные
data = pd.read_csv('Data/StateFarm.csv', sep=';')

# создаем массив меток и массив признаков
y = data.pop('Response')

# создаем экземпляр класса GradientBoostingClassifier
boost = GradientBoostingClassifier(random_state=46)

# выполняем перекрестную проверку и сохраняем результат 
# с помощью функции cross_validate()
output = cross_validate(boost, 
                        data, 
                        y, 
                        cv=5, 
                        return_estimator=True)

In [25]:
# создаем список fi, в который будем сохранять 
# важности признаков, и сохраняем в него важности, 
# рассчитанные для каждой из моделей
fi = []
for estimator in output['estimator']:
    fi.append(estimator.feature_importances_)

In [26]:
# преобразовываем список в датафрейм, индексы в котором 
# будут именами наших переменных
fi = pd.DataFrame(
    np.array(fi).T,
    columns=['importance ' + str(idx) for idx in range(len(fi))],
    index=data.columns)
fi

,importance 0,importance 1,importance 2,importance 3,importance 4
Customer Lifetime Value,0.255318,0.234372,0.269660,0.203805,0.161001
Income,0.348616,0.332639,0.296688,0.313324,0.361096
Monthly Premium Auto,0.139044,0.154778,0.167606,0.142860,0.183078
Months Since Last Claim,0.073406,0.084058,0.092329,0.116752,0.092812
Months Since Policy Inception,0.125063,0.121127,0.114898,0.142917,0.115305
Number of Open Complaints,0.025870,0.030423,0.026967,0.031941,0.038452
Number of Policies,0.032684,0.042602,0.031851,0.048403,0.048257


In [27]:
# получаем усредненные важности 
# и выводим в порядке убывания
fi['mean_importance'] = fi.mean(axis=1)
fi.sort_values('mean_importance', ascending=False)

,importance 0,importance 1,importance 2,importance 3,importance 4,mean_importance
Income,0.348616,0.332639,0.296688,0.313324,0.361096,0.330472
Customer Lifetime Value,0.255318,0.234372,0.269660,0.203805,0.161001,0.224831
Monthly Premium Auto,0.139044,0.154778,0.167606,0.142860,0.183078,0.157473
Months Since Policy Inception,0.125063,0.121127,0.114898,0.142917,0.115305,0.123862
Months Since Last Claim,0.073406,0.084058,0.092329,0.116752,0.092812,0.091871
Number of Policies,0.032684,0.042602,0.031851,0.048403,0.048257,0.040759
Number of Open Complaints,0.025870,0.030423,0.026967,0.031941,0.038452,0.030731
